In [1]:
import pandas as pd
import nltk
import re
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

#Thiết lập các thông số khác theo nhu cầu
vectorizer = CountVectorizer(min_df=1)

In [3]:
f_data = pd.read_csv('./remove_dup_nan.csv')
f_data

,Unnamed: 0,name,category,description,time,URL,priceLow,priceHigh,reviews,adress,rating
0,0,Veteran Restaurant,"Steakhouse, Barbecue, Asian",Xin Chao. We are Stone Grill Barbecue Restaura...,"['Sun:5:00 PM - 12:00 AM', 'Mon:5:00 PM - 12:0...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,"['Terrible service. We came there many times, ...","8 Duong Dinh Nghe, Da Nang 550000 Vietnam",4.0
1,1,Cardi Pizzeria Bach Dang,"American, Steakhouse, Brew Pub",Premium Steak & Pizza Certified Imported Steak...,"['Sun:12:00 AM - 11:59 PM', 'Mon:12:00 AM - 11...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,['I came in for pizza because they look like t...,"124 Bạch Đằng, Hải Châu, Đà Nẵng, Da Nang Vietnam",5.0
2,2,Ca Chuon Co Vietnamese - Seafood Restaurant.,"Cafe, International, Asian","Ca Chuon Co, ('flying fish') a modern stylish ...","['Sun:06:00 AM - 10:00 PM', 'Mon:06:00 AM - 10...",https://www.tripadvisor.com//Restaurant_Review...,"₩12,953","₩38,860","[""I was looking for some decent seafoods resta...",99 Vo Nguyen Giap Street Ngu Hanh Son District...,5.0
3,3,Bistecca Restaurant Danang,"Italian, Southwestern, European",Bistecca – Italian restaurant offering the ori...,"['Sun:06:00 AM - 10:00 PM', 'Mon:06:00 AM - 10...",https://www.tripadvisor.com//Restaurant_Review...,"₩3,886","₩64,767",['Beautiful dinner and great service! All meal...,"20 Dong Da 7F New Orient Hotel, Thuan Phuoc Wa...",5.0
4,4,3 Big - Nướng & Lẩu,"Pizza, Barbecue, Korean",3 BiG - Nướng & Lẩu Nhà hàng BBQ - Thịt nướng ...,"['Sun:11:00 AM - 11:00 PM', 'Mon:11:00 AM - 10...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,['Great service. Good price and good food. The...,"134 Phan Châu Trinh, Phước Ninh, Hải Châu, Đà ...",5.0
...,...,...,...,...,...,...,...,...,...,...,...
1012,1117,SOHO Buffet & Alacarte Restaurant,"Japanese, Seafood, Barbecue",As the world of food up to nearly 300 dishes i...,"['Sun:10:30 AM - 2:00 PM:5:30 PM - 10:00 PM', ...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,['Staff need to improve the service & english ...,"Lot 1, Street 2/9, Da Nang 550000 Vietnam",2.5
1013,1119,East West Brewing Co. – Da Nang,"American, Seafood, International, Vietnamese",NaN,"['Sun:09:00 AM - 11:00 PM', 'Mon:09:00 AM - 11...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,['The service was terrible and the food was ev...,"1A Võ Nguyên Giáp, Phước Mỹ, Sơn Trà, Đà Nẵng,...",2.0
1014,1124,Danang Tower Steakhouse & Soju Bar,"Steakhouse, Seafood, Asian",From the mouthwatering seafood to the tender g...,NaN,https://www.tripadvisor.com//Restaurant_Review...,"₩11,654","₩87,403","[""Nice view. The first time I have been here w...","Đ. Trần Hưng Đạo 37/F, Vinpearl Condotel River...",1.5
1015,1127,Burger King,"Quick Bites, Fast Food",NaN,NaN,https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,['This Burger King is outrageously expensive a...,"Da Nang Airport Hai Chau Dist, Da Nang Vietnam",2.0


In [4]:
revd = f_data['reviews']
revd

0       ['Terrible service. We came there many times, ...
1       ['I came in for pizza because they look like t...
2       ["I was looking for some decent seafoods resta...
3       ['Beautiful dinner and great service! All meal...
4       ['Great service. Good price and good food. The...
                              ...                        
1012    ['Staff need to improve the service & english ...
1013    ['The service was terrible and the food was ev...
1014    ["Nice view. The first time I have been here w...
1015    ['This Burger King is outrageously expensive a...
1016    ['I really love food here. I will come back he...
Name: reviews, Length: 1017, dtype: object

In [5]:
#f_data.loc[800]

In [6]:
#f_data.loc[1048]

In [7]:
#f_data[f_data.duplicated(keep = False)]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# revd = revd.fillna('0')

count_vect_category = CountVectorizer(min_df=0.0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(revd) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [10]:
# Biên độ vector tính năng văn bản giữa các dữ liệu đánh giá văn bản
revd = revd.dropna()

count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(revd) 


# Đánh giá sự tương đồng giữa các review text
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [11]:
# in không lược bớt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [16]:
# Công thức 1. Độ tương đồng loại, 2. Độ tương đồng của review text
place_simi_co = (
                 + place_simi_cate * 0.2
                 + place_simi_review * 0.8
                )

place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 

def find_simi_place(f_data, sorted_ind, place_name, top_n=10):
    
    if place_name not in f_data['name'].values:
        print('Viết nhầm tên cửa hàng rồi. Viết lại đi')
    else:
        place_title = f_data[f_data['name'] == place_name]
        place_index = place_title.index.values
        similar_indexes = sorted_ind[place_index, :(top_n)]
        similar_indexes = similar_indexes.reshape(-1)
        return f_data.iloc[similar_indexes]

find_simi_place(f_data, place_simi_co_sorted_ind,'Ca Chuon Co Vietnamese - Seafood Restaurant.', 6)

,Unnamed: 0,name,category,description,time,URL,priceLow,priceHigh,reviews,adress,rating
2,2,Ca Chuon Co Vietnamese - Seafood Restaurant.,"Cafe, International, Asian","Ca Chuon Co, ('flying fish') a modern stylish ...","['Sun:06:00 AM - 10:00 PM', 'Mon:06:00 AM - 10...",https://www.tripadvisor.com//Restaurant_Review...,"₩12,953","₩38,860","[""I was looking for some decent seafoods resta...",99 Vo Nguyen Giap Street Ngu Hanh Son District...,5.0
31,31,Sky View Restaurant,"International, Healthy, Vegetarian Friendly",NaN,"['Sun:11:00 AM - 11:00 PM', 'Mon:11:00 AM - 11...",https://www.tripadvisor.com//Restaurant_Review...,₩3,₩48,['Good service . The food so good . \nThe staf...,"216 Vo Nguyen Giap, Phuoc My Ward, Son Tra Dis...",5.0
34,35,Bep Cuon Da Nang,"Asian, Vietnamese, Healthy",NaN,"['Sun:10:30 AM - 9:00 PM', 'Mon:10:30 AM - 9:0...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,['Went to this restaurant as it rated highly o...,"54 Nguyen Van Thoai Ngu Hanh Son, Da Nang 5500...",5.0
37,38,Cape Nao Beach Club,"Seafood, Mediterranean, European",Whilst enjoying the preciousness of this heave...,"['Sun:11:00 AM - 11:00 PM', 'Mon:11:00 AM - 11...",https://www.tripadvisor.com//Restaurant_Review...,"₩7,752","₩64,599",['I did not stay here. But I had lunch outside...,"Melia Danang Beach Resort, No.19, Truong Sa st...",5.0
65,66,Sofia Restaurant,"Asian, Vietnamese, Vegetarian Friendly","Da Nang Sofia Restaurant, being to become one ...","['Sun:06:00 AM - 10:00 PM', 'Mon:06:00 AM - 10...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,"['Cute place, excellent food, fruit juices and...","Slot I - 11, Pham Van Dong Near My Khe Beach, ...",4.5
5,5,Thìa Gỗ Restaurant Da Nang,"Vietnamese, Vegetarian Friendly",NaN,"['Sun:10:00 AM - 10:00 PM', 'Mon:10:00 AM - 10...",https://www.tripadvisor.com//Restaurant_Review...,"₩2,124","₩26,554",['Quynh was our server and took great care of ...,"53 Phan Thuc Duyen, My An, Da Nang 50507 Vietnam",5.0


In [13]:
#f_data.loc[800]

In [14]:
#f_data.loc[1048]

In [15]:
#f_data.loc[1294]